In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import featurewiz as gwiz

Imported 0.3.0 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [2]:
data = pd.read_csv('dataset.csv')
data.head()


,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


In [3]:
data.SMILES.count()

9982

In [4]:
data.Solubility.count()

9982

In [5]:
#MessingAround
#Chem.MolFromSmiles(data.SMILES[0])
#Chem.MolFromSmiles('ClCC(Cl)(Cl)Cl')
#m = Chem.MolFromSmiles('ClCC(Cl)(Cl)Cl')
#m.GetNumAtoms()

Calculating molecular descriptors in rdkit

In [6]:
mol_list= []
for element in data.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not removing hydrogen atom without neighbors
[17:01:33] WARNING: not r

In [7]:
len(mol_list)

9982

In [8]:
mol_list[:5]

https://github.com/dataprofessor/code/blob/master/python/cheminformatics_predicting_solubility.ipynb
Calculate molecular descriptors
To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:
cLogP (Octanol-water partition coefficient)
MW (Molecular weight)
RB (Number of rotatable bonds)
AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)
Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, we will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute.

LogP, MW and RB

In [9]:
def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [10]:
df = generate(data.SMILES)
df

[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not removing hydrogen atom without neighbors
[17:01:35] WARNING: not r

,MolLogP,MolWt,NumRotatableBonds
0,3.95810,392.510,17.0
1,2.40550,169.183,0.0
2,2.15250,140.569,1.0
3,8.11610,756.226,10.0
4,2.48540,422.525,12.0
...,...,...,...
9977,2.61700,264.369,8.0
9978,-0.21440,444.440,2.0
9979,2.82402,150.221,1.0
9980,5.09308,454.611,13.0


Aromatic Proportion

In [11]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')     

In [12]:
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms

[False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True]

In [13]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [14]:
AromaticAtoms(m)

19

In [15]:
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
desc_AromaticAtoms

[0,
 10,
 6,
 36,
 12,
 6,
 0,
 0,
 12,
 12,
 6,
 6,
 36,
 6,
 6,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 6,
 16,
 0,
 0,
 12,
 6,
 6,
 0,
 0,
 6,
 0,
 28,
 12,
 30,
 12,
 9,
 0,
 28,
 28,
 0,
 0,
 6,
 0,
 14,
 6,
 6,
 6,
 0,
 0,
 0,
 12,
 0,
 12,
 0,
 0,
 0,
 6,
 0,
 0,
 24,
 0,
 6,
 6,
 0,
 12,
 12,
 24,
 0,
 18,
 6,
 0,
 12,
 0,
 0,
 0,
 10,
 0,
 30,
 0,
 24,
 0,
 0,
 6,
 26,
 0,
 0,
 0,
 18,
 12,
 0,
 0,
 0,
 0,
 0,
 6,
 24,
 12,
 0,
 0,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 18,
 16,
 6,
 0,
 12,
 0,
 24,
 6,
 12,
 0,
 24,
 0,
 6,
 6,
 0,
 26,
 6,
 0,
 26,
 0,
 0,
 6,
 0,
 0,
 9,
 6,
 6,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 12,
 0,
 0,
 18,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 6,
 0,
 0,
 54,
 6,
 0,
 0,
 12,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 12,
 0,
 12,
 6,
 12,
 0,
 0,
 0,
 22,
 0,
 12,
 18,
 6,
 12,
 0,
 6,
 6,
 0,
 0,
 6,
 0,
 36,
 6,
 0,
 6,
 0,
 12,
 0,
 0,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 10,
 0,
 6,
 38,


No. of heavy atoms

In [16]:
#single molecule
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
Descriptors.HeavyAtomCount(m)

34

In [17]:
#all dataset
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_HeavyAtomCount

[23,
 13,
 9,
 53,
 31,
 9,
 12,
 27,
 16,
 25,
 11,
 14,
 54,
 15,
 18,
 11,
 14,
 35,
 11,
 16,
 14,
 10,
 10,
 7,
 13,
 31,
 10,
 10,
 27,
 18,
 2,
 14,
 35,
 13,
 35,
 27,
 16,
 12,
 1,
 14,
 11,
 9,
 69,
 26,
 49,
 28,
 16,
 11,
 63,
 58,
 12,
 18,
 21,
 21,
 15,
 11,
 14,
 13,
 14,
 12,
 10,
 20,
 29,
 21,
 63,
 13,
 5,
 9,
 16,
 26,
 42,
 13,
 15,
 24,
 2,
 16,
 25,
 42,
 10,
 21,
 14,
 15,
 28,
 10,
 9,
 7,
 11,
 10,
 54,
 3,
 52,
 14,
 5,
 16,
 42,
 12,
 16,
 27,
 33,
 16,
 12,
 13,
 8,
 13,
 14,
 20,
 46,
 17,
 19,
 19,
 156,
 21,
 28,
 18,
 22,
 12,
 45,
 16,
 58,
 19,
 64,
 18,
 16,
 20,
 14,
 23,
 34,
 40,
 13,
 22,
 63,
 56,
 3,
 19,
 13,
 37,
 44,
 10,
 4,
 42,
 14,
 10,
 11,
 1,
 2,
 13,
 21,
 10,
 11,
 7,
 11,
 3,
 23,
 7,
 11,
 13,
 9,
 16,
 20,
 22,
 31,
 25,
 32,
 22,
 26,
 6,
 13,
 43,
 14,
 3,
 24,
 11,
 12,
 13,
 5,
 24,
 15,
 13,
 12,
 100,
 11,
 12,
 16,
 27,
 13,
 14,
 11,
 13,
 10,
 6,
 16,
 14,
 34,
 9,
 20,
 14,
 17,
 17,
 5,
 1,
 28,
 16,
 13,
 35,
 15,
 3

Aromatic Proportion Descriptor 

In [18]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
AromaticAtoms(m)/Descriptors.HeavyAtomCount(m)

0.5588235294117647

In [19]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_AromaticProportion

[0.0,
 0.7692307692307693,
 0.6666666666666666,
 0.6792452830188679,
 0.3870967741935484,
 0.6666666666666666,
 0.0,
 0.0,
 0.75,
 0.48,
 0.5454545454545454,
 0.42857142857142855,
 0.6666666666666666,
 0.4,
 0.3333333333333333,
 0.0,
 0.42857142857142855,
 0.0,
 0.5454545454545454,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.46153846153846156,
 0.0,
 0.6,
 0.0,
 0.0,
 0.0,
 0.0,
 0.42857142857142855,
 0.45714285714285713,
 0.0,
 0.0,
 0.4444444444444444,
 0.375,
 0.5,
 0.0,
 0.0,
 0.5454545454545454,
 0.0,
 0.4057971014492754,
 0.46153846153846156,
 0.6122448979591837,
 0.42857142857142855,
 0.5625,
 0.0,
 0.4444444444444444,
 0.4827586206896552,
 0.0,
 0.0,
 0.2857142857142857,
 0.0,
 0.9333333333333333,
 0.5454545454545454,
 0.42857142857142855,
 0.46153846153846156,
 0.0,
 0.0,
 0.0,
 0.6,
 0.0,
 0.5714285714285714,
 0.0,
 0.0,
 0.0,
 0.6666666666666666,
 0.0,
 0.0,
 0.5714285714285714,
 0.0,
 0.4,
 0.25,
 0.0,
 0.75,
 0.48,
 0.5714285714285714,
 0.0,
 0.8571428571428571,
 0.42857142857142855,

In [20]:
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])
df_desc_AromaticProportion

,AromaticProportion
0,0.000000
1,0.769231
2,0.666667
3,0.679245
4,0.387097
...,...
9977,0.315789
9978,0.187500
9979,0.545455
9980,0.363636


X matrix (Combining all computed descriptors into 1 dataframe)

In [21]:
X = pd.concat([df,df_desc_AromaticProportion], axis=1)
X

,MolLogP,MolWt,NumRotatableBonds,AromaticProportion
0,3.95810,392.510,17.0,0.000000
1,2.40550,169.183,0.0,0.769231
2,2.15250,140.569,1.0,0.666667
3,8.11610,756.226,10.0,0.679245
4,2.48540,422.525,12.0,0.387097
...,...,...,...,...
9977,2.61700,264.369,8.0,0.315789
9978,-0.21440,444.440,2.0,0.187500
9979,2.82402,150.221,1.0,0.545455
9980,5.09308,454.611,13.0,0.363636


In [22]:
data.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


Y-Matrix

In [23]:
Y = data.iloc[:,1]
Y

0              N,N,N-trimethyloctadecan-1-aminium bromide
1                                Benzo[cd]indol-2(1H)-one
2                                    4-chlorobenzaldehyde
3       zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...
4       4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...
                              ...                        
9977                                           tetracaine
9978                                         tetracycline
9979                                               thymol
9980                                            verapamil
9981                                             warfarin
Name: Name, Length: 9982, dtype: object

DataSplit

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [26]:
Y_train.head()

2351    methyl(naphthalen-1-ylmethyl)amine
603            2,2,2-trichloroacetaldehyde
4831                    p-cyclohexylphenol
7866                4-isothiocyanatophenol
6826                      2-chlorobiphenyl
Name: Name, dtype: object

LINREG

In [27]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [28]:
df.head()

,MolLogP,MolWt,NumRotatableBonds
0,3.9581,392.510,17.0
1,2.4055,169.183,0.0
2,2.1525,140.569,1.0
3,8.1161,756.226,10.0
4,2.4854,422.525,12.0


# Instantiate the OneHotEncoder object
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')

# Apply one-hot encoding to the categorical column
encoded = encoder.fit_transform(data[['Name']])

# Convert the encoded data back to a data frame
encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names())

# Print the encoded data frame
print(encoded_df)

HELP!

In [29]:
model = linear_model.LinearRegression()
#model.fit(X_train, Y_train)